# Answering Business Questions Using SQL

In this project, we'll use SQL to answer some typical business questions using the Chinook database.

A copy of the `chinook.db` schema can be seen below:

![chinook-schema](chinook-schema.svg)

### Setup

You can use the following code if you need to install `ipython-sql`:

    !conda install -yc conda-forge ipython-sql
    
First, we'll connect our Jupyter Notebook to our database file.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

Next, let's take a first look at the database and list out the tables and views to get a better idea of what we're working with.

In [2]:
%%sql

SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


Here is our scenario:

The Chinook Record Store has signed a deal with a new record label, and we need to select the first 3 out of 4 albums that we will add to the store. We only have the artists names and genre of music they produce as follows:


| Artist Name | Genre |
| :--- | --- |
| Regal | Hip-Hop |
|Red Tone | Punk |
|Meteor and the Girls | Pop |
|Slim Jim Bites | Blues |


### Selecting Albums to Purchase

First, we'll find out which genres sell best in the US to help us effectively choose which albums to advertise. 

In [3]:
%%sql

WITH usa_sold AS
   (SELECT il.*
      FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     WHERE i.billing_country = 'USA'
   )

SELECT g.name genre,
       COUNT(us.quantity) tracks_sold,
       ROUND(CAST(COUNT(us.quantity) AS FLOAT) / (
           SELECT COUNT(*) FROM usa_sold
       ) * 100, 1) percentage_sold
  FROM usa_sold us  
 INNER JOIN track t ON t.track_id = us.track_id
 INNER JOIN genre g ON g.genre_id = t.genre_id
 GROUP BY genre
 ORDER BY tracks_sold DESC
 LIMIT 15;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Pop,22,2.1
Latin,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


Looking at the best selling genres in the US we determine that we should purchase 

| Artist Name | Genre | % Sold |
| :--- | --- | --- |
|Red Tone | Punk | 12.4 |
|Slim Jim Bites | Blues | 3.4 |
|Meteor and the Girls | Pop | 2.1 |

We will not purchase Regal's Hip Hop album since Hip Hop/Rap only makes up 1.9% of sales by genre.

Something to note however is that Rock makes up a massive 53.4% of albums sold, so we should be on the lookout for any new artists in this genre.


### Analyzing Employee Sales Performance


Next, we'll look at the purchases of customers to see which of the sales support agents are the best performers.

In [4]:
%%sql

WITH customer_support_rep AS
     (
     SELECT i.customer_id,
            c.support_rep_id,
            SUM(i.total) AS total
       FROM invoice i
      INNER JOIN customer c ON i.customer_id = c.customer_id
      GROUP BY i.customer_id, c.support_rep_id
     )
    
SELECT 
       e.first_name || " " || e.last_name AS employee,
       e.hire_date,
       CAST(SUM(csr.total) AS INT) AS total_sales
  FROM customer_support_rep csr
 INNER JOIN employee e ON e.employee_id = csr.support_rep_id
 GROUP BY employee;

 * sqlite:///chinook.db
Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731
Margaret Park,2017-05-03 00:00:00,1584
Steve Johnson,2017-10-17 00:00:00,1393


There doesn't seem to be a significant difference between the top-performing employee, and the lowest-performing employee, given that the total sales of each employee seems to roughly correspond to how long they've been employed.

### Analyzing Sales by Country

Next, we'll analyze the sales for customers from each coutry. We'll calculate the total number of customers, total value of sales, average value of sales per customer, and average order value. For the countries that only have one customer, we'll group those customers into a separate "Other" category. Note that for this case we've been asked to use `country` from the customer table, and not `billing_country` from the invoice_line table.

In [7]:
%%sql

WITH countries_and_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                   FROM customer
                  WHERE country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
        END AS country,
       c.customer_id,
       il.*
      FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )


SELECT country,
       customers,
       total_sales,
       average_order,
       customer_lifetime_value
FROM
       (
       SELECT country,
              COUNT(DISTINCT customer_id) customers,
              SUM(unit_price) total_sales,
              SUM(unit_price) / COUNT(DISTINCT customer_id) customer_lifetime_value,
              SUM(unit_price) / COUNT(DISTINCT invoice_id) average_order,
         CASE
              WHEN country = "Other" THEN 1
              ELSE 0
          END AS sort
         FROM countries_and_other
        GROUP BY country
        ORDER BY sort ASC, total_sales DESC
       );

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


In [11]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM countries_summed
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        ROUND(SUM(unit_price), 2) total_sales,
        ROUND(SUM(unit_price) / count(distinct customer_id), 2) customer_lifetime_value,
        ROUND(SUM(unit_price) / count(distinct invoice_id), 2) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.57
India,2,183.15,8.72,91.58
Other,15,1094.94,7.45,73.0


In [14]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       il.*
     FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
    ),
    
countries_summed AS
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        ROUND(SUM(unit_price), 2) total_sales,
        ROUND(SUM(unit_price) / count(distinct customer_id), 2) customer_lifetime_value,
        ROUND(SUM(unit_price) / count(distinct invoice_id), 2) average_order,
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    )
    
SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM countries_summed;

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.49,7.94,80.04
Canada,8,535.59,7.05,66.95
Brazil,5,427.68,7.01,85.54
France,5,389.07,7.78,77.81
Germany,4,334.62,8.16,83.66
Czech Republic,2,273.24,9.11,136.62
United Kingdom,3,245.52,8.77,81.84
Portugal,2,185.13,6.38,92.57
India,2,183.15,8.72,91.58
Other,15,1094.94,7.45,73.0


Findings

### Albums vs Individual Tracks

Findings.

The idea for this project comes from the [DATAQUEST](https://app.dataquest.io/) **Intermediate SQL for Data Analysis** course.